# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [20]:
import pandas as pd

In [21]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**. 

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method. 

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [ ]:
# orders
orders.duplicated().sum()

0

In [ ]:
# orderlines
orderlines.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [ ]:
orders.info()

* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [ ]:
orderlines.info()

* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [ ]:
print(f"5 missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

A quick way to find out a percentage here, if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [ ]:
orders.total_paid.isna().value_counts(normalize=True)

False    0.999978
True     0.000022
Name: total_paid, dtype: float64

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [4]:
orders = orders.loc[~orders.total_paid.isna(), :]

Should you have a significant number of missing values in the future, you have a choice: 
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [ ]:
orders["created_date"] = pd.to_datetime(orders["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [ ]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

#### 4.1.2.&nbsp;`unit_price`

In [ ]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [ ]:
orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [ ]:
two_dot_percentage = ((orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [ ]:
two_dot_order_ids_list = orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines = orderlines.loc[~orderlines.id_order.isin(two_dot_order_ids_list)]

In [ ]:
orderlines.shape[0]

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [ ]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [6]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

### Look for Duplicates

In [ ]:
# your code here
products.duplicated().sum()
len(products)

19326

In [ ]:
# check for multiple SKU's
# not necessary because empty prices are droppped

#products.groupby("sku").agg({"name": "count"}).sort_values(by='name')
#products[products['sku']=='APP1197']

### Drop Duplicates

In [ ]:
products = products.drop_duplicates()
len(products)

10580

In [ ]:
# remove empty duplicate row
# products = products.drop(labels=8000, axis=0)

### Look for Missing values


In [ ]:
# your code here
products.info()

* desc 7 missing/NaN
* price contains 46 NaN 
* type 50 missing/NaN
*
* price is object
* promo_price is object

### Dealing with missing values

In [ ]:
products.desc.isna().value_counts(normalize=True)

99.9% of descriptions (desc) contain valid value -> desc are not essential for analysis can be neglected and left empty

In [ ]:
products.price.isna().value_counts(normalize=True)

False    0.995652
True     0.004348
Name: price, dtype: float64

99.5% of prices contain valid value (is 46 values) -> How are tables connected? can prices be found in orderlist when connecting with prices using sku?

In [ ]:
products.type.isna().value_counts(normalize=True)

99.5% of types contain valid value -> only 50 types missing could be done manually but for now we will drop it and come back to it later

### Drop NaN


In [7]:
products = products.loc[~products.type.isna(), :]

In [8]:
products = products.loc[~products.price.isna(), :]

In [9]:
products = products.loc[~products.desc.isna(), :]

### Check / Change Data types

In [ ]:
products

In [ ]:
# Ana's solution
products_1 = products.loc[products.price.str.contains("\d+\.\d+\.\d+"), :]
products_1['price'] = products_1.price.str.replace(".","").astype(float)
products_1['price'] = products_1['price'].div(1000)
products_1.head(30)

In [ ]:
products.promo_price.str.contains("^\d+\.\d+$").value_counts(normalize=True) 

In [ ]:
#products.promo_price.str.contains("\.\d+$").value_counts(normalize=True) 
#products[products.promo_price.str.contains("\.\d\d\d$")]
#products[products.promo_price.str.contains("^\d\.\d$")]
#products[products.price.str.contains("^\d+\.\d+\.\d+$")]

In [ ]:
# price 2x "."
products_2d = products.loc[products.price.str.contains("\d+\.\d+\.\d+"), :]
# price 1x "."
products_1d = products.loc[products.price.str.contains("^\d+\.\d+$"), :]
# price 0x "."
products_0d = products.loc[products.price.str.contains("^\d+$"), :]

In [ ]:
# price 1x".", 3 digits after
# if price and promo price are equal delete last digit
products_1do_2di = products.loc[products.price.str.contains("^\d+\.\d\d\d$"), :]
products_1do_2di
for i in range(len(products_1do_2di)):
  if products_1do_2di["price"][i] == products_1do_2di["promo_price"][i]:
    products_1do_2di["price"][i] = products_1do_2di["price"][i][:-1] # string kürzen

# how often price != promo_price
#products_1do_2di[(products_1do_2di["type"]== "1,44E+11") & (products_1do_2di["price"] != products_1do_2di["promo_price"])]#.sort_values(by="sku").head(30)
#products_1do_2di[products_1do_2di["price"] != products_1do_2di["promo_price"]].groupby("type").count().sort_values(by="sku").head(30)

In [ ]:
import re

prod_30 = products.head(30)
pattern = re.compile("^\d+\.\d\d\d$")

for i in range(len(prod_30)):
  if prod_30["price"][i].str.contains("^\d+\.\d\d\d$"):
    if prod_30["price"][i] == prod_30["promo_price"][i]:
      prod_30["price"][i] = prod_30["price"][i][:-1]
      prod_30["promo_price"][i] = prod_30["promo_price"][i][:-1]
    

In [ ]:
prod_30 = products.tail(30)
prod_30

In [ ]:
import re
i =0
#prod_30.index(prod_30["price"].str.contains("^\d+\.\d\d\d$"))

pattern = re.compile("^\d+\.\d\d\d$")
pattern2 = re.compile("\d+\.\d+\.\d+")

for i in range(len(prod_30)):
  if pattern.search(prod_30["price"][i]) is not None:
    if prod_30["price"][i] == prod_30["promo_price"][i]:
      prod_30["price"][i] = prod_30["price"][i][:-1]
      prod_30["promo_price"][i] = prod_30["promo_price"][i][:-1]
  elif pattern2.search(prod_30["price"][i]) is not None:
    print(i)

prod_30

In [ ]:
# price 2x".", 3 digits after
#products_2do_2di = products.loc[products.price.str.contains("\d+\.\d+\.\d\d\d"), :]

products_2do_2di = products_2do_2di.head(30)
split_prod = products_2do_2di["price"].str.split(".") # price

#for i in range(len(prod_30)):
  #print(products_2do_2di.iloc[i, 3][:-1])

split_prod

In [ ]:
split_prod_whole = products["price"].str.split(".") # price
len(split_prod_whole)
#split_prod_whole
#split_prod_whole[0] # whole value split up at "."
#split_prod_whole[0][0] # singel value - number of digits
i = 100
for i in range(len(split_prod_whole)): # whole table len
  if len(split_prod_whole[i]) == 3:
      print("3", i)
  elif len(split_prod_whole[i]) == 2:
      print("2", i)
  elif len(split_prod_whole[i]) == 1:
      print("1", i)
  else:
    print(len(split_prod_whole[i]))

#len(split_prod_whole[0])

In [ ]:
products_2do_2di

In [ ]:
# how often is price != promo_price
products[products["price"] != products["promo_price"]].groupby("type").count().sort_values(by="sku",ascending=False).head(30)

## Solution 

### remove promo_price column

In [10]:
products = products.drop("promo_price", axis=1)

In [ ]:
products.sample(30)

## remove price rows with wrong format

In [12]:
products = products.loc[~products.price.str.contains("^\d+\.\d\d\d$"), :]

In [13]:
products = products.loc[~products.price.str.contains("\d+\.\d+\.\d+"), :]

In [ ]:
price_problems_number = products.loc[(products.price.str.contains("\d+\.\d+\.\d+"))|(products.price.str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

In [15]:
products["price"] = pd.to_numeric(products["price"])

In [17]:
products_cl = products

## Save files

In [19]:
from google.colab import files

orders.to_csv("orders_cl.csv", index=False)
files.download("orders_cl.csv")

orderlines.to_csv("orderlines_cl.csv", index=False)
files.download("orderlines_cl.csv")

products_cl.to_csv("products_cl.csv", index=False)
files.download("products_cl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>